In [ ]:
from iterator import SmartIterator
from utils.visualization_utils import get_att_map, objdict, get_dict
from old_models import ReferringRelationshipsModel
from utils.eval_utils import iou_bbox

from sklearn.metrics import roc_auc_score
from keras import backend as K
import numpy as np
import os
from PIL import Image
import json
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
########### METRICS #########################################

def sim_metric_np(y_true, y_pred, eps=10e-8):
    y_true = (y_true.T/(eps + y_true.sum(axis=1).T)).T
    y_pred = (y_pred.T/(eps + y_pred.sum(axis=1).T)).T
    mini = ((y_true*(y_true<y_pred)) + (y_pred*(y_pred<y_true))).sum(axis=1)
    return list(mini)

def iou_np(y_true, y_pred, thresh=0.5, eps=10e-8):
    y_pred = y_pred > thresh
    intersection = (y_pred * y_true).sum(axis=1)
    union = eps + ((y_pred + y_true)>0).sum(axis=1)
    return list(intersection/union)

def recall_np(y_true, y_pred, thresh=0.5, eps=10e-8):
    y_pred = y_pred > thresh
    tp = (y_pred * y_true).sum(axis=1)
    fn = (1*((y_true - y_pred)>0)).sum(axis=1)
    recall = tp/(tp+fn+eps)
    return list(recall)

def precision_np(y_true, y_pred, thresh=0.5, eps=10e-8):
    y_pred = y_pred > thresh
    tp = (y_pred * y_true).sum(axis=1)
    p = y_pred.sum(axis=1)
    prec = tp/(p+eps)
    return list(prec)

def kl_metric_np(y_true, y_pred, eps=10e-8):
    y_true = (y_true.T/(eps + y_true.sum(axis=1).T)).T
    y_pred = (y_pred.T/(eps + y_pred.sum(axis=1).T)).T
    x = np.log(eps+(y_true/(eps+y_pred)))
    return list((x*y_true).sum(axis=1))

def cc_metric_np(y_true, y_pred, eps=10e-8):
    sigma_true = y_true.var(axis=1)
    sigma_pred = y_pred.var(axis=1)
    cov = ((y_true-y_true.mean(axis=1, keepdims=True)) * (y_pred - y_pred.mean(axis=1, keepdims=True))).mean(axis=1)
    return list(cov/np.sqrt((sigma_true*sigma_true)+eps))

def iou_bbox_np(y_true, y_pred, thresh=0.5, eps=10e-8):
    gt_bbox = get_bbox_from_heatmap(y_true, thresh)
    pred_bbox = get_bbox_from_heatmap(y_pred, thresh)
    return list(iou(gt_bbox, pred_bbox))

def roc_auc(y_true, y_pred):
    scores = []
    for i in range(y_true.shape[0]):
        if y_true[i].sum()>0:
            scores += [roc_auc_score(y_true[i], y_pred[i], average='micro')]
    return scores

def pixel_acc(y_true, y_pred, thresh=0.5):
    y_pred = y_pred > thresh
    acc = (y_pred == y_true).mean(axis=1)
    return list(acc)

########### HELPERS #########################################

def get_bbox_from_heatmap(heatmap, threshold, input_dim=224):
    heatmap = heatmap.reshape((-1, input_dim, input_dim)) 
    heatmap[heatmap < threshold] = 0
    horiz = 1. * (heatmap.sum(axis=2, keepdims=True)>0)
    horiz = horiz.repeat(input_dim, axis=2)
    vert = 1. * (heatmap.sum(axis=1, keepdims=True)>0)
    vert = vert.repeat(input_dim, axis=1)
    mask = horiz * vert
    return mask

def iou(y_true, y_pred, eps=10e-8):
    intersection = (y_pred * y_true).sum(axis=1)
    union = eps + ((y_pred + y_true)>0).sum(axis=1)
    return intersection/union

In [ ]:
def load_model(model_checkpoint):
    params = objdict(json.load(open(os.path.join(os.path.dirname(model_checkpoint), "args.json"), "r")))
    params.finetune_cnn = True
    model_weights = h5py.File(model_checkpoint)
    relationships_model = ReferringRelationshipsModel(params)
    model = relationships_model.build_model()
    model.load_weights(model_checkpoint)
    return model, params
    

def evaluate_model(model, params, test_data_dir, batch_size, metrics=[iou_np, recall_np, precision_np, kl_metric_np, cc_metric_np, sim_metric_np, pixel_acc]):
    params.batch_size = batch_size
    params.shuffle = False
    test_generator = SmartIterator(test_data_dir, params)
    results = {}
    for metric in metrics:
        results[metric.__name__+'_s'] = []
        results[metric.__name__+'_o'] = []
    for i in range(len(test_generator)):
        if i%10 == 0:
            print("{}/{}".format(i, len(test_generator)))
        batch_in, batch_out = test_generator[i]
        preds = model.predict(batch_in)
        for metric in metrics:
            results[metric.__name__+'_s'] += metric(batch_out[0], preds[0])
            results[metric.__name__+'_o'] += metric(batch_out[1], preds[1])
    final= ""
    for metric in metrics:
        mean_s = np.mean(results[metric.__name__+'_s'])
        mean_o = np.mean(results[metric.__name__+'_o'])
        print("{} : {:.4f} & {:.4f} ".format(metric.__name__, mean_s, mean_o))
        final += " {:.4f} & {:.4f} ".format(mean_s, mean_o)
    print(final)
    return results

# VRD 

In [ ]:
test_data_dir = "/data/ranjaykrishna/ReferringRelationships/data/dataset-vrd-14/test"
batch_size = 49 # or 140 or 49 or 245,

## Baseline no predicate

In [ ]:
model_checkpoint = "/data/chami/ReferringRelationships/models/final_models/VisualGenome/baseline_no_predicate/model08-1.96.h5"
model, params = load_model(model_checkpoint)
results = evaluate_model(model, params, test_data_dir, batch_size)

## Baseline + predicate

In [ ]:
model_checkpoint = "/data/chami/ReferringRelationships/models/VRD/11_13_2017/baseline/1/model08-3.92.h5"
model, params = load_model(model_checkpoint)
results = evaluate_model(model, params, test_data_dir, batch_size)

## SSAN (iter=1)

In [ ]:
model_checkpoint = "/data/chami/ReferringRelationships/models/final_models/VRD/sym_ssn_1/model09-2.77.h5"
model, params = load_model(model_checkpoint)
results = evaluate_model(model, params, test_data_dir, batch_size)

## SSAN (iter=2)

In [ ]:
model_checkpoint = "/data/chami/ReferringRelationships/models/final_models/VRD/sym_ssn_2/model07-2.49.h5"
model, params = load_model(model_checkpoint)
results = evaluate_model(model, params, test_data_dir, batch_size)

## SSAN (iter=3)

In [ ]:
model_checkpoint = "/data/chami/ReferringRelationships/models/final_models/VRD/sym_ssn_3/model07-2.30.h5"
model, params = load_model(model_checkpoint)
results = evaluate_model(model, params, test_data_dir, batch_size)

# Visual Genome 

In [ ]:
test_data_dir = "/data/ranjaykrishna/ReferringRelationships/data/dataset-visualgenome-14/test"
batch_size = 245 # or 140 or 49, 6863 samples

## Baseline

In [ ]:
model_checkpoint = "/data/chami/ReferringRelationships/models/final_models/VisualGenome/"
model, params = load_model(model_checkpoint)
results = evaluate_model(model, params, test_data_dir, batch_size)

## Baseline + Predicate

In [ ]:
model_checkpoint = "/data/chami/ReferringRelationships/models/final_models/VisualGenome/baseline/model07-1.77.h5"
model, params = load_model(model_checkpoint)
results = evaluate_model(model, params)

In [ ]:
model_checkpoint = "/data/chami/ReferringRelationships/models/final_models/VRD/sym_ssn_3/model07-2.30.h5"
model, params = load_model(model_checkpoint)
results = evaluate_model(model, params)

## SSAN (iter=1)

In [ ]:
model_checkpoint = "/data/chami/ReferringRelationships/models/final_models/VRD/sym_ssn_3/model07-2.30.h5"
model, params = load_model(model_checkpoint)
results = evaluate_model(model, params)

## SSAN (iter=2)

In [ ]:
model_checkpoint = "/data/chami/ReferringRelationships/models/final_models/VRD/sym_ssn_3/model07-2.30.h5"
model, params = load_model(model_checkpoint)
results = evaluate_model(model, params)

## SSAN (iter=3)

In [ ]:
model_checkpoint = "/data/chami/ReferringRelationships/models/final_models/VRD/sym_ssn_3/model07-2.30.h5"
model, params = load_model(model_checkpoint)
results = evaluate_model(model, params)

In [ ]:
def evaluate_model_per_cat(model, params, metrics=[iou_np, recall_np, precision_np, roc_auc]):
    test_data_dir = "/data/ranjaykrishna/ReferringRelationships/data/dataset-vrd-14/test"
    params.batch_size = 140
    categories = json.load(open('data/VRD/objects.json', 'r'))
    params.baseline_weights = None
    params.shuffle = False
    test_generator = SmartIterator(test_data_dir, params)
    results = {}
    for i in range(len(categories)):
        results[i] = {}
        for metric in metrics:
            results[i][metric.__name__+'_s'] = []
            results[i][metric.__name__+'_o'] = []
    for i in range(len(test_generator)):
        if i%10 == 0:
            print("{}/{}".format(i, len(test_generator)))
        batch_in, batch_out = test_generator[i]
        preds = model.predict(batch_in)
        for j in range(len(categories)):
            indices = batch_in[1] == j
            if indices.sum() > 0:
                sub_pred = [preds[0][indices.flatten()], preds[1][indices.flatten()]]
                sub_batch_out = [batch_out[0][indices.flatten()], batch_out[1][indices.flatten()]]
                for metric in metrics:
                    results[j][metric.__name__+'_s'] += metric(sub_batch_out[0], sub_pred[0])
            indices = batch_in[2] == j
            if indices.sum() > 0:
                sub_pred = [preds[0][indices.flatten()], preds[1][indices.flatten()]]
                sub_batch_out = [batch_out[0][indices.flatten()], batch_out[1][indices.flatten()]]
                for metric in metrics:
                    results[j][metric.__name__+'_o'] += metric(sub_batch_out[1], sub_pred[1])
    for i in results.keys():
        for j in results[i].keys():
            results[i][j] = np.mean(results[i][j])
    return results

In [ ]:
for cat in results_cat:
    "Subject IOU per cat"
    print("{} : {}".format(categories[cat], results_cat[cat]["precision_np_s"]))

In [ ]:
for cat in results_cat:
    "Object IOU per cat"
    print("{} : {}".format(categories[cat], results_cat[cat]["precision_np_o"]))

In [ ]:
model_checkpoint = "/data/chami/ReferringRelationships/models/VRD/11_07_2017/sym_ssn/1/model18-1.66.h5"
#model_checkpoint = "/data/chami/ReferringRelationships/models/VRD/11_07_2017/sym_ssn/3/model21-0.96.h5"
#model_checkpoint = "/data/chami/ReferringRelationships/models/VRD/11_07_2017/sym_ssn/2/model17-1.37.h5"
model, params = load_model(model_checkpoint)
results = evaluate_model(model, params)

In [ ]:
model_checkpoint = "/data/chami/ReferringRelationships/models/VRD/11_07_2017/ssn/5/model19-2.03.h5"
model, params = load_model(model_checkpoint)
preds = evaluate_model(model, params)

In [ ]:
model_checkpoint = "/data/chami/ReferringRelationships/models/VRD/11_07_2017/baseline/7/model28-1.51.h5"
model, params = load_model(model_checkpoint)
preds = evaluate_model(model, params)

In [ ]:
model_checkpoint = "/data/chami/ReferringRelationships/models/VRD/11_07_2017/baseline_no_predicate/8/model24-1.38.h5"
model, params = load_model(model_checkpoint)
preds = evaluate_model(model, params)

### VRD Models

In [ ]:
# models selected with val iou
best_baseline_checkpoint = "/data/chami/ReferringRelationships/models/VRD/10_27_2017/baseline/8/model12-0.51.h5"
best_baseline_no_pred_checkpoint = "/data/chami/ReferringRelationships/models/VRD/10_27_2017/baseline_no_predicate/22/model36-0.53.h5"
best_ssn_checkpoint = "/data/chami/ReferringRelationships/models/VRD/10_27_2017/ssn/3/model35-0.54.h5"
best_sym_ssn_checkpoint = "/data/chami/ReferringRelationships/models/VRD/10_27_2017/sym_ssn/21/model11-0.52.h5"
best_sym_ssn_internal_loss_checkpoint = "/data/chami/ReferringRelationships/models/VRD/10_31_2017/1/model18-1.06.h5"

# models selected with val iou bbox
#best_baseline_checkpoint = "/data/chami/ReferringRelationships/models/VRD/10_27_2017/baseline/3/model17-0.50.h5"
#best_baseline_no_pred_checkpoint = '/data/chami/ReferringRelationships/models/VRD/10_27_2017/baseline_no_predicate/3/model25-0.51.h5'
#best_ssn_checkpoint = '/data/chami/ReferringRelationships/models/VRD/10_27_2017/ssn/22/model12-0.55.h5'
#best_sym_ssn_checkpoint = '/data/chami/ReferringRelationships/models/VRD/10_27_2017/sym_ssn/12/model17-0.54.h5'

# models selected with val_loss
#best_baseline_checkpoint = "/data/chami/ReferringRelationships/models/VRD/10_27_2017/baseline/2/model34-0.49.h5"
#best_baseline_no_pred_checkpoint = '/data/chami/ReferringRelationships/models/VRD/10_27_2017/baseline_no_predicate/3/model31-0.50.h5'
#best_ssn_checkpoint = '/data/chami/ReferringRelationships/models/VRD/10_27_2017/ssn/19/model47-0.50.h5'
#best_sym_ssn_checkpoint = '/data/chami/ReferringRelationships/models/VRD/10_27_2017/sym_ssn/5/model47-0.50.h5'


## Test Results 

In [ ]:
import math

def divisorGenerator(n):
    large_divisors = []
    for i in range(1, int(math.sqrt(n) + 1)):
        if n % i == 0:
            yield i
            if i*i != n:
                large_divisors.append(n / i)
    for divisor in reversed(large_divisors):
        yield divisor
        
list(divisorGenerator(6862))